In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/iris/Iris.csv
/kaggle/input/iris/database.sqlite


In [2]:
import torch
import torch.optim as optim
import torch.nn as nn

In [3]:
import torch.nn.init as init
class Net(nn.Module):
    def __init__(self,n_features=4):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(n_features,100)
        # Batch normalization after output
        #self.bn1 = nn.BatchNorm1d(100)
        self.fc2 = nn.Linear(100,50)
        #self.bn2 = nn.BatchNorm1d(50)
        self.fc3 = nn.Linear(50,3)
        # Applying weight initialization techniques
        init.kaiming_uniform_(self.fc1.weight)
        init.kaiming_uniform_(self.fc2.weight)
        init.kaiming_uniform_(self.fc3.weight, nonlinearity ="sigmoid")
    def forward(self,inputs):
        x = self.fc1(inputs)
        #x = self.bn1(x)
        x = nn.functional.elu(x)
        x = self.fc2(x)
        #x = self.bn2(x)
        x = nn.functional.elu(x)
        x = self.fc3(x)
        return x
    

In [4]:
net = Net()

In [5]:
from torch.utils.data import TensorDataset, DataLoader, random_split
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split

class IrisDataset:
    def __init__(self, data, targets, batch_size=32):
        """
        Initialize the dataset with data, targets, and batch size.
        """
        # Store data and targets as TensorDataset
        self.dataset = TensorDataset(torch.tensor(data, dtype=torch.float32), 
                                     torch.tensor(targets, dtype=torch.long))
        self.batch_size = batch_size

    def get_splits(self, train_ratio=0.7, val_ratio=0.15):
        """
        Split the dataset into train, validation, and test sets.
        """
        total_size = len(self.dataset)
        train_size = int(train_ratio * total_size)
        val_size = int(val_ratio * total_size)
        test_size = total_size - train_size - val_size

        # Use random_split to create the splits
        train_set, val_set, test_set = random_split(
            self.dataset, [train_size, val_size, test_size]
        )

        return train_set, val_set, test_set

    def get_dataloaders(self, train_set, val_set, test_set):
        """
        Create DataLoaders for each split.
        """
        train_loader = DataLoader(train_set, batch_size=self.batch_size, shuffle=True,drop_last = True)
        val_loader = DataLoader(val_set, batch_size=self.batch_size, shuffle=False)
        test_loader = DataLoader(test_set, batch_size=self.batch_size, shuffle=False)

        return train_loader, val_loader, test_loader


In [6]:
import pandas as pd
df = pd.read_csv('/kaggle/input/iris/Iris.csv')

In [7]:
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
features = df.iloc[:,1:5].to_numpy()
features = scaler.fit_transform(features)
targets,_ = pd.factorize(df.iloc[:,5])
#Convert to tensors for target
targets = F.one_hot(torch.tensor(targets),num_classes=3)

In [8]:
features[0]

array([-0.90068117,  1.03205722, -1.3412724 , -1.31297673])

In [9]:
iris_dataset = IrisDataset(features, targets, batch_size=4)

    # Get train, validation, and test splits
train_set, val_set, test_set = iris_dataset.get_splits()

    # Create DataLoaders for each split
train_loader, val_loader, test_loader = iris_dataset.get_dataloaders(train_set, val_set, test_set)


/tmp/ipykernel_17/4132533968.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(targets, dtype=torch.long))


In [10]:
next(iter(train_loader))

[tensor([[ 0.7957, -0.5878,  0.4784,  0.3962],
         [-0.1737,  1.7263, -1.1707, -1.1815],
         [ 0.3110, -0.3564,  0.5353,  0.2647],
         [ 0.1898,  0.8007,  0.4216,  0.5276]]),
 tensor([[0, 1, 0],
         [1, 0, 0],
         [0, 1, 0],
         [0, 1, 0]])]

In [11]:
from torch.nn import CrossEntropyLoss
criterion = CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = 1e-3)
num_epochs = 50

In [12]:
def train(model,num_epochs,criterion,optimizer,dataloader):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0.0
        batch_count = 0
        for batch in dataloader:
            optimizer.zero_grad()
            #forward pass
            features,labels = batch
            preds = model(features)
            #print("Predictions = ", preds)
            #Calculate loss
            loss = criterion(preds,labels.argmax(dim = -1))
            #backward pass
            loss.backward()
            #Update model weights
            optimizer.step()
            #calculate mean loss
            total_loss += loss.item()
            batch_count +=1
        mean_loss = total_loss/batch_count
        print(f"Epoch : {epoch+1}, mean loss : {mean_loss}")

In [13]:
len(train_loader)

26

In [14]:
train(net,100,criterion,optimizer,train_loader)

Epoch : 1, mean loss : 0.6820293994477162
Epoch : 2, mean loss : 0.3197017124352547
Epoch : 3, mean loss : 0.2512810413654034
Epoch : 4, mean loss : 0.18725086877552363
Epoch : 5, mean loss : 0.1492792537364249
Epoch : 6, mean loss : 0.12152757609706336
Epoch : 7, mean loss : 0.09932168045914015
Epoch : 8, mean loss : 0.09230511134060529
Epoch : 9, mean loss : 0.08335785428509833
Epoch : 10, mean loss : 0.07060415885755972
Epoch : 11, mean loss : 0.0652409644222084
Epoch : 12, mean loss : 0.06186674135992339
Epoch : 13, mean loss : 0.05445261108868111
Epoch : 14, mean loss : 0.05510119931736531
Epoch : 15, mean loss : 0.05453064917674055
Epoch : 16, mean loss : 0.044079740021306164
Epoch : 17, mean loss : 0.04875964710887414
Epoch : 18, mean loss : 0.04165300543536432
Epoch : 19, mean loss : 0.0407784567192161
Epoch : 20, mean loss : 0.04415127632651442
Epoch : 21, mean loss : 0.036624022100183465
Epoch : 22, mean loss : 0.03318937283564502
Epoch : 23, mean loss : 0.04310176677664948
E